# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-00886fcba1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Monarchies") 

Consider the following exploratory information need:

> You investigating existing and past historical monarchies

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P1366`   | replaced by    | predicate |
| `wdt:P122`    | basic form of government    | predicate |
| `wd:Q38`      | Italy          | node |
| `wd:Q145`     | United Kingdom           | node |
| `wd:Q7269`    | monarchy           | node |




Also consider

```
?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
```

is the BGP to retrieve all **countries that are some instance of a monarchy**

## Workload Goals


1. Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

2. Identify the BGP for obtaining statistical information of monarchies, e.g., poluation or inception date

3. Which is or was  the country with the highest population?

4. How many countries are or have been monarchies across continents?

5. Analyze the number of countries that were monarchies
 
   5.1 How many sub-types of monarchy exist?  How many instances of exist of each one?
   
   5.2 Are there cities that have been capital of multiple kingdoms?
   
   5.3 Which are the top-3 monarchies with the earliest inception and still existing?


In [2]:
# start your workflow here

In [3]:
    queryString = """
    SELECT COUNT(*)
    WHERE { 

        ?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    } 
    """

    print("Results")
    run_query(queryString)

Results
[('callret-0', '2344')]


1

# Workload Goals

##### Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

In [4]:
# wd:Q7269	monarchy	node
queryString = """
SELECT *
WHERE { 

    wd:Q7269 ?p ?o .   
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9486'), ('o', '3545'), ('pname', 'HERO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('o', '300055529'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('o', '8373'), ('pname', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('o', 'JC375'), ('pname', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('o', 'http://www.wikidata.org/entity/Q8209976'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0124448'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('o', 'topic/monarchy'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Heraldic%20Royal%20Crown%20%28Common%29.svg'), ('pname', 'image')]
[('p', 'http

50

In [5]:
# wd:Q7269	monarchy	node
queryString = """
SELECT *
WHERE { 

    ?p ?o wd:Q7269 .   
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "ir", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1183407'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Taifa of Algeciras')]
[('p', 'http://www.wikidata.org/entity/Q484195'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Timurid Empire')]
[('p', 'http://www.wikidata.org/entity/Q215443'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Swedish Empire')]
[('p', 'http://www.wikidata.org/entity/Q12150341'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Samanid Empire')]
[('p', 'http://www.wikidata.org/entity/Q215530'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Kingdom of Ireland')]
[('p', 'http://www.wikidata.org/entity/Q93180'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Seleucid Empire')]
[('p', 'http://www.wikidata.org/entity/Q205718'), ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Zirid dynasty')]
[('p', 'http://www.wikidata.org/entity/Q733897'), ('o', 'http://www.wiki

43

In [6]:
# [('p', 'http://www.wikidata.org/entity/Q18234383'), 
# ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Safavid Empire')]

# [('p', 'http://www.wikidata.org/entity/Q83311'), 
# ('o', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'Persian Empire')]

queryString = """
SELECT *
WHERE { 
    ?p ?o wd:Q83311 .   
        # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ir", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5762342'), ('o', 'http://www.wikidata.org/prop/direct/P1001'), ('pname', 'Koi'), ('oname', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'Iran'), ('oname', 'replaces')]
[('p', 'http://www.wikidata.org/entity/Q1969109'), ('o', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'Persian windmill'), ('oname', 'named after')]
[('p', 'http://www.wikidata.org/entity/Q10394208'), ('o', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'chess in Persia'), ('oname', 'country')]
[('p', 'http://www.wikidata.org/entity/Q3273852'), ('o', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'Hellespontine Phrygia'), ('oname', 'country')]
[('p', 'http://www.wikidata.org/entity/Q1163924'), ('o', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'Celaenae'), ('oname', 'country')]
[('p', 'http://www.wikidata.org/entity/Q102104649'), ('o', 'http://www.wik

50

In [7]:
# [('p', 'http://www.wikidata.org/entity/Q18234383'), 
# ('o', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'Safavid Empire')]

# [('p', 'http://www.wikidata.org/entity/Q83311'), 
# ('o', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'Persian Empire')]

queryString = """
SELECT *
WHERE { 
    ?p ?o wd:Q83311 .   
        # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "iran", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'Iran'), ('oname', 'replaces')]


1

In [8]:
# [('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'),
# ('pname', 'Iran'), ('oname', 'replaces')]
queryString = """
SELECT *
WHERE { 
    wd:Q794 wdt:P1365 ?monarchy .   
        # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q207991'), ('monarchyname', 'Pahlavi Dynasty')]
[('monarchy', 'http://www.wikidata.org/entity/Q83311'), ('monarchyname', 'Persian Empire')]
[('monarchy', 'http://www.wikidata.org/entity/Q107258515'), ('monarchyname', 'Pahlavi Iran')]


3

In [9]:
queryString = """
SELECT *
WHERE { 
    ?p ?o wd:Q207991 .   
        # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER(REGEX(STR(?oname), "^[Rr][eplace]"))
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
[('p', 'http://www.wikidata.org/entity/Q4767446'), ('o', 'http://www.wikidata.org/prop/direct/P551'), ('oname', 'residence')]


3

In [10]:
queryString = """
SELECT *
WHERE { 
    wd:Q7269 ?p ?o .   
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER(REGEX(STR(?oname), "^[Rr][eplace]"))
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9486'), ('o', '3545'), ('pname', 'HERO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('o', '300055529'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('o', '8373'), ('pname', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('o', 'JC375'), ('pname', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('o', 'http://www.wikidata.org/entity/Q8209976'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0124448'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('o', 'topic/monarchy'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Heraldic%20Royal%20Crown%20%28Common%29.svg'), ('pname', 'image')]
[('p', 'http

50

In [11]:
queryString = """
SELECT *
WHERE { 
    ?monarchy ?o wd:Q7269 .   
        # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?o <http://schema.org/name> ?oname .
    FILTER NOT EXISTS { ?monarchy wdt:P122 wd:Q7269 } 
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q101423800'), ('o', 'http://www.wikidata.org/prop/direct/P106'), ('monarchyname', 'Jacob Gyang Buba'), ('oname', 'occupation')]
[('monarchy', 'http://www.wikidata.org/entity/Q3640980'), ('o', 'http://www.wikidata.org/prop/direct/P1142'), ('monarchyname', 'National Bloc of Freedom'), ('oname', 'political ideology')]
[('monarchy', 'http://www.wikidata.org/entity/Q2993893'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('monarchyname', 'royal council'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q5186811'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('monarchyname', 'criticism of monarchy'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q7995869'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('monarchyname', 'abolition of monarchy'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q106570346'), ('o', 'http://www.wikidata.org/prop/direct/P1269'

20

In [12]:
queryString = """
SELECT *
WHERE { 
    ?monarchy ?o wd:Q7269 .   
        # Labels
    #?monarchy <http://schema.org/name> ?monarchyname .
    ?o <http://schema.org/name> ?oname .
    FILTER NOT EXISTS { ?monarchy wdt:P122 wd:Q7269 } 
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q101423800'), ('o', 'http://www.wikidata.org/prop/direct/P106'), ('oname', 'occupation')]
[('monarchy', 'http://www.wikidata.org/entity/Q3640980'), ('o', 'http://www.wikidata.org/prop/direct/P1142'), ('oname', 'political ideology')]
[('monarchy', 'http://www.wikidata.org/entity/Q2993893'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q5186811'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q7995869'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q106570346'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity/Q60788337'), ('o', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of')]
[('monarchy', 'http://www.wikidata.org/entity

20

In [13]:
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 
    ?monarchy ?o wd:Q7269 .   
        # Labels
    #?monarchy <http://schema.org/name> ?monarchyname .
    ?o <http://schema.org/name> ?oname .
    #FILTER NOT EXISTS { ?monarchy wdt:P122 wd:Q7269 } 
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/prop/direct/P106'), ('oname', 'occupation')]
[('o', 'http://www.wikidata.org/prop/direct/P1142'), ('oname', 'political ideology')]
[('o', 'http://www.wikidata.org/prop/direct/P122'), ('oname', 'basic form of government')]
[('o', 'http://www.wikidata.org/prop/direct/P1269'), ('oname', 'facet of')]
[('o', 'http://www.wikidata.org/prop/direct/P180'), ('oname', 'depicts')]
[('o', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
[('o', 'http://www.wikidata.org/prop/direct/P301'), ('oname', "category's main topic")]
[('o', 'http://www.wikidata.org/prop/direct/P31'), ('oname', 'instance of')]
[('o', 'http://www.wikidata.org/prop/direct/P360'), ('oname', 'is a list of')]
[('o', 'http://www.wikidata.org/prop/direct/P361'), ('oname', 'part of')]
[('o', 'http://www.wikidata.org/prop/direct/P4224'), ('oname', 'category contains')]
[('o', 'http://www.wikidata.org/prop/direct/P527'), ('oname', 'has part')]
[('o', 'http://www.wikidata.org/

16

In [14]:
queryString = """
SELECT *
WHERE { 
    ?monarchy wdt:P31 wd:Q7269 .   
        # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?o <http://schema.org/name> ?oname .
    #?p <http://schema.org/name> ?pname .
    #FILTER NOT EXISTS { ?monarchy wdt:P122 wd:Q7269 } 
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q2196956'), ('monarchyname', 'Kingdom of Norway (872–1397)')]
[('monarchy', 'http://www.wikidata.org/entity/Q83311'), ('monarchyname', 'Persian Empire')]
[('monarchy', 'http://www.wikidata.org/entity/Q669459'), ('monarchyname', 'Monarchy of Australia')]
[('monarchy', 'http://www.wikidata.org/entity/Q3244497'), ('monarchyname', 'monarchy of the Bahamas')]
[('monarchy', 'http://www.wikidata.org/entity/Q703340'), ('monarchyname', 'Wiman Joseon')]
[('monarchy', 'http://www.wikidata.org/entity/Q6898103'), ('monarchyname', 'Monarchy of South Africa')]
[('monarchy', 'http://www.wikidata.org/entity/Q508840'), ('monarchyname', 'Monarchy of Canada')]
[('monarchy', 'http://www.wikidata.org/entity/Q10331792'), ('monarchyname', 'Portuguese monarchy')]
[('monarchy', 'http://www.wikidata.org/entity/Q4296'), ('monarchyname', 'monarchy of New Zealand')]
[('monarchy', 'http://www.wikidata.org/entity/Q16840097'), ('monarchyname', 'monarchy of Greece')

20

In [15]:
# [('o', 'http://www.wikidata.org/prop/direct/P360'), ('oname', 'is a list of')]
queryString = """
SELECT *
WHERE { 
    ?monarchy wdt:P360 wd:Q7269 .   
        # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?o <http://schema.org/name> ?oname .
    #FILTER NOT EXISTS { ?monarchy wdt:P122 wd:Q7269 } 
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q3494968'), ('monarchyname', 'monarchies in Europe')]
[('monarchy', 'http://www.wikidata.org/entity/Q3446184'), ('monarchyname', 'list of monarchies')]
[('monarchy', 'http://www.wikidata.org/entity/Q3510397'), ('monarchyname', 'Monarchies in Africa')]


3

In [16]:
# [('o', 'http://www.wikidata.org/prop/direct/P360'), ('oname', 'is a list of')]
queryString = """
SELECT *
WHERE { 
    wd:Q3446184 ?p ?o .  
    
        # Labels
    #?monarchy <http://schema.org/name> ?monarchyname .
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
    #FILTER NOT EXISTS { ?monarchy wdt:P122 wd:Q7269 } 
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('o', 'http://www.wikidata.org/entity/Q8791896'), ('oname', 'Category:Monarchies'), ('pname', 'category related to list')]
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('o', 'http://www.wikidata.org/entity/Q54068'), ('oname', 'Category:Monarchy'), ('pname', 'category related to list')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oname', 'Wikimedia list article'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('o', 'http://www.wikidata.org/entity/Q7269'), ('oname', 'monarchy'), ('pname', 'is a list of')]


4

In [17]:
queryString = """
SELECT *
WHERE { 
    ?monarchy wdt:P122 wd:Q7269 .   
        # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?o <http://schema.org/name> ?oname .
    #FILTER(REGEX(STR(?monarchyname), "[Ss]afavid"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q139315'), ('monarchyname', 'Dʿmt')]
[('monarchy', 'http://www.wikidata.org/entity/Q30673322'), ('monarchyname', 'Safavid Georgia')]
[('monarchy', 'http://www.wikidata.org/entity/Q30890672'), ('monarchyname', 'Kingdom of Sicily')]
[('monarchy', 'http://www.wikidata.org/entity/Q153080'), ('monarchyname', 'East Francia')]
[('monarchy', 'http://www.wikidata.org/entity/Q449639'), ('monarchyname', 'Alodia')]
[('monarchy', 'http://www.wikidata.org/entity/Q733668'), ('monarchyname', 'Ghassanids')]
[('monarchy', 'http://www.wikidata.org/entity/Q1183407'), ('monarchyname', 'Taifa of Algeciras')]
[('monarchy', 'http://www.wikidata.org/entity/Q1389743'), ('monarchyname', 'Kotoko kingdom')]
[('monarchy', 'http://www.wikidata.org/entity/Q696640'), ('monarchyname', 'Duchy of Pomerania')]
[('monarchy', 'http://www.wikidata.org/entity/Q3110559'), ('monarchyname', 'Pauravas')]
[('monarchy', 'http://www.wikidata.org/entity/Q484195'), ('monarchyname',

40

In [18]:
### retrieve all **countries that are some instance of a monarchy**
queryString = """
SELECT ?p ?pname
WHERE { 

    ?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
            # Labels
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q139315'), ('pname', 'Dʿmt')]
[('p', 'http://www.wikidata.org/entity/Q139315'), ('pname', 'Dʿmt')]
[('p', 'http://www.wikidata.org/entity/Q688859'), ('pname', 'Duchy of Bouillon')]
[('p', 'http://www.wikidata.org/entity/Q15883591'), ('pname', 'Anhalt-Dessau')]
[('p', 'http://www.wikidata.org/entity/Q2308237'), ('pname', 'Ahom Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q2308237'), ('pname', 'Ahom Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q3919363'), ('pname', 'Principality of Lahojsk')]
[('p', 'http://www.wikidata.org/entity/Q6207845'), ('pname', 'Jodhpur State')]
[('p', 'http://www.wikidata.org/entity/Q1740533'), ('pname', 'Baku Khanate')]
[('p', 'http://www.wikidata.org/entity/Q13032661'), ('pname', 'Principality of Slutsk')]
[('p', 'http://www.wikidata.org/entity/Q625380'), ('pname', 'Eastern Hungarian Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q896856'), ('pname', 'Prince-Bishopric Konstanz')]
[('p', 'http://www.wi

20

In [19]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?monarchy ?monarchyname ?replacesname ?replacedbyname
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q8733'), ('monarchyname', 'Qing dynasty'), ('replacesname', 'Da Ming Shun Tian Guo'), ('replacedbyname', 'Da Ming Shun Tian Guo')]
[('monarchy', 'http://www.wikidata.org/entity/Q8733'), ('monarchyname', 'Qing dynasty'), ('replacesname', 'Da Ming Shun Tian Guo'), ('replacedbyname', 'Republic of China 1912–1949')]
[('monarchy', 'http://www.wikidata.org/entity/Q11916665'), ('monarchyname', 'Datarpur'), ('replacesname', 'Guler State'), ('replacedbyname', 'India')]
[('monarchy', 'http://www.wikidata.org/entity/Q11949691'), ('monarchyname', 'Siba State'), ('replacesname', 'Guler State'), ('replacedbyname', 'India')]
[('monarchy', 'http://www.wikidata.org/entity/Q11916665'), ('monarchyname', 'Datarpur'), ('replacesname', 'Guler State'), ('replacedbyname', 'Provinces of India')]
[('monarchy', 'http://www.wikidata.org/entity/Q11949691'), ('monarchyname', 'Siba State'), ('replacesname', 'Guler State'), ('replacedbyname', 'Provinces of India')

40

In [20]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?p ?pname ?o
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby ?p ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q42351257')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q42351257')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q7269')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q7269')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q55502')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q12536')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q7887194')]
[('p', 'http://www.wikidata.org/prop/direct/P1

40

In [21]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?p ?pname ?o
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby ?p ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER(REGEX(STR(?pname), "[Pp]opulation"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '2437574')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '11984911')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '10400000')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '10400000')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '785240')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '676376')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '3190619')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '2590939')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '2278361')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '2942436')]
[('p', 'http://www.wikidata.org

40

In [22]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT count (DISTINCT ?p)
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby ?p ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '544')]


1

In [23]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT DISTINCT ?p ?pname ?o
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby ?p ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q79')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q43287')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q15102440')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q516160')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q3025345')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q3540212')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q965692')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q8575586')]
[('p', 'http://w

40

In [24]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT COUNT (DISTINCT ?pname)
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby ?p ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2')]


1

In [25]:
#[('p', 'http://www.wikidata.org/entity/Q794'), ('o', 'http://www.wikidata.org/prop/direct/P1365'), ('oname', 'replaces')]
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT DISTINCT ?pname
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby ?p ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('pname', 'country')]
[('pname', 'country calling code')]


2

In [26]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT DISTINCT ?o
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8733')]
[('o', 'http://www.wikidata.org/entity/Q668')]
[('o', 'http://www.wikidata.org/entity/Q8680')]
[('o', 'http://www.wikidata.org/entity/Q41304')]
[('o', 'http://www.wikidata.org/entity/Q43287')]
[('o', 'http://www.wikidata.org/entity/Q7318')]
[('o', 'http://www.wikidata.org/entity/Q131964')]
[('o', 'http://www.wikidata.org/entity/Q34266')]
[('o', 'http://www.wikidata.org/entity/Q696241')]
[('o', 'http://www.wikidata.org/entity/Q38')]
[('o', 'http://www.wikidata.org/entity/Q191077')]
[('o', 'http://www.wikidata.org/entity/Q29')]
[('o', 'http://www.wikidata.org/entity/Q843')]
[('o', 'http://www.wikidata.org/entity/Q79')]
[('o', 'http://www.wikidata.org/entity/Q45670')]
[('o', 'http://www.wikidata.org/entity/Q377350')]
[('o', 'http://www.wikidata.org/entity/Q1028')]
[('o', 'http://www.wikidata.org/entity/Q33')]
[('o', 'http://www.wikidata.org/entity/Q15102440')]
[('o', 'http://www.wikidata.org/entity/Q3025345')]
[('o', 'http://www.wiki

40

In [27]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT COUNT (DISTINCT ?o)
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?o .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '109')]


1

In [28]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?monarchyname ?countryname
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Eastern Hungarian Kingdom'), ('countryname', 'Principality of Transylvania')]
[('monarchyname', 'East Francia'), ('countryname', 'Holy Roman Empire')]
[('monarchyname', 'Province of North Carolina'), ('countryname', 'United States of America')]
[('monarchyname', 'Song dynasty'), ('countryname', 'Yuan Empire')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania')]
[('monarchyname', 'Travancore'), ('countryname', 'India')]
[('monarchyname', 'Travancore'), ('countryname', 'India')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Kingdom of Cochin'), ('countryname', 'India')]
[('monarchyname', 'Kingdom of C

40

In [29]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?monarchyname ?countryname
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Eastern Hungarian Kingdom'), ('countryname', 'Principality of Transylvania')]
[('monarchyname', 'East Francia'), ('countryname', 'Holy Roman Empire')]
[('monarchyname', 'Province of North Carolina'), ('countryname', 'United States of America')]
[('monarchyname', 'Song dynasty'), ('countryname', 'Yuan Empire')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania')]
[('monarchyname', 'Travancore'), ('countryname', 'India')]
[('monarchyname', 'Travancore'), ('countryname', 'India')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Kingdom of Cochin'), ('countryname', 'India')]
[('monarchyname', 'Kingdom of C

40

In [30]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?monarchyname ?countryname ?country
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Eastern Hungarian Kingdom'), ('countryname', 'Principality of Transylvania'), ('country', 'http://www.wikidata.org/entity/Q655621')]
[('monarchyname', 'East Francia'), ('countryname', 'Holy Roman Empire'), ('country', 'http://www.wikidata.org/entity/Q12548')]
[('monarchyname', 'Province of North Carolina'), ('countryname', 'United States of America'), ('country', 'http://www.wikidata.org/entity/Q30')]
[('monarchyname', 'Song dynasty'), ('countryname', 'Yuan Empire'), ('country', 'http://www.wikidata.org/entity/Q7313')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania'), ('country', 'http://www.wikidata.org/entity/Q203493')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania'), ('country', 'http://www.wikidata.org/entity/Q203493')]
[('monarchyname', 'Travancore'), ('countryname', 'India'), ('country', 'http://www.wikidata.org/entity/Q668')]
[('monarchyname', 'Travancore'), ('countryname', 'India'), ('

40

In [31]:
# 
# http://www.wikidata.org/entity/Q794
queryString = """
SELECT *
WHERE { 

    wd:Q794 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "begin", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [32]:
# 
# http://www.wikidata.org/entity/Q794
queryString = """
SELECT *
WHERE { 

    wd:Q794 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "1979", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [33]:
# 
# http://www.wikidata.org/entity/Q794
queryString = """
SELECT COUNT (DISTINCT ?p)
WHERE { 

    wd:Q794 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '205')]


1

In [34]:
# find the time 
# http://www.wikidata.org/entity/Q794
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

    wd:Q794 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('pname', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1174'), ('pname', 'visitors per year')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pname', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1304'), ('pname', 'central bank')]


10

In [35]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?monarchyname ?monarchy ?countryname ?country
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    FILTER(REGEX(STR(?monarchyname), "^[Pp]ahlavi"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), ('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]


1

In [36]:
# monarchy time 
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 ?p ?o .
    wd:Q207991 wdt:P31 ?instance .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?instance <http://schema.org/name> ?instancename .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q164950'), ('instancename', 'dynasty')]
[('instance', 'http://www.wikidata.org/entity/Q3024240'), ('instancename', 'historical country')]
[('instance', 'http://www.wikidata.org/entity/Q6256'), ('instancename', 'country')]


3

In [37]:
# monarchy time 
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 ?p ?o .

    wd:Q3024240 wdt:P373 ?o . 

    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'Former countries')]


1

In [38]:
# [('instance', 'http://www.wikidata.org/entity/Q3024240'), ('instancename', 'historical country')]
# monarchy time 
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 ?p ?o .

    wd:Q3024240 ?p ?o . 

    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

### start date, date or time 

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('o', 'http://www.wikidata.org/entity/Q28864179'), ('pname', 'different from'), ('oname', 'proposed country')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q19953632'), ('pname', 'subclass of'), ('oname', 'former administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P580'), ('pname', 'inappropriate property for this type'), ('oname', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P582'), ('pname', 'inappropriate property for this type'), ('oname', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('o', 'http://www.wikidata.org/entity/P576'), ('pname', 'properties for this type'), ('oname', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('o', 'http://www.wikidata.org/entity/P17'), ('pnam

14

In [39]:
# ('oname', 'dissolved, abolished or demolished date')]
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 ?p ?o .

    wd:Q3024240 ?p ?o . 

    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

    FILTER(REGEX(STR(?oname), ".[Dd]ate")) 
    
    #FILTER(REGEX(STR(?oname), "[Tt]ime")) 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('o', 'http://www.wikidata.org/entity/P576'), ('pname', 'properties for this type'), ('oname', 'dissolved, abolished or demolished date')]


1

In [40]:
# ('oname', 'dissolved, abolished or demolished date')]
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 ?p ?o .

    wd:Q3024240 ?p ?o . 

    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

    #FILTER(REGEX(STR(?oname), ".[Dd]ate")) 
    
    FILTER(REGEX(STR(?oname), "[Tt]ime")) 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P580'), ('pname', 'inappropriate property for this type'), ('oname', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P582'), ('pname', 'inappropriate property for this type'), ('oname', 'end time')]


2

In [41]:
# historical country wd:Q3024240 ?p ?o . 
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 wdt:P1963 ?o .

    wd:Q3024240 wdt:P1963 ?o . 

    # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

    #FILTER(REGEX(STR(?oname), ".[Dd]ate")) 
    
    #FILTER(REGEX(STR(?oname), "[Tt]ime")) 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P576'), ('oname', 'dissolved, abolished or demolished date')]
[('o', 'http://www.wikidata.org/entity/P571'), ('oname', 'inception')]
[('o', 'http://www.wikidata.org/entity/P36'), ('oname', 'capital')]
[('o', 'http://www.wikidata.org/entity/P1365'), ('oname', 'replaces')]
[('o', 'http://www.wikidata.org/entity/P1366'), ('oname', 'replaced by')]
[('o', 'http://www.wikidata.org/entity/P3842'), ('oname', 'located in present-day administrative territorial entity')]


6

In [42]:
# historical country wd:Q3024240 ?p ?o . 
# 'inception' P571
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 wdt:P1963 ?o .
    
    wd:Q3024240 wdt:P1963 ?o .
    ?o ?p ?s .

    # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

    FILTER(REGEX(STR(?oname), "[Ii]nception")) 
    
    #FILTER(REGEX(STR(?oname), "[Tt]ime")) 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('s', 'http://wikiba.se/ontology#Property'), ('oname', 'inception')]
[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1628'), ('s', 'http://purl.org/dc/terms/created'), ('oname', 'inception')]
[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1628'), ('s', 'http://id.nlm.nih.gov/mesh/vocab#dateCreated'), ('oname', 'inception')]
[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1628'), ('s', 'http://schema.org/startDate'), ('oname', 'inception')]
[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1628'), ('s', 'https://schema.org/dateCreated'), ('oname', 'inception')]
[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1628'), ('s', 'https://schema.org/foundingDate'), ('oname'

20

In [43]:
# historical country wd:Q3024240 ?p ?o . 
# 'inception' P571
queryString = """
SELECT *
WHERE { 

    #wd:Q207991 wdt:P1963 ?o .
    
    wd:Q3024240 ?o wd:P571 .

    # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

    #FILTER(REGEX(STR(?oname), "[Ii]nception")) 
    
    #FILTER(REGEX(STR(?oname), "[Tt]ime")) 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/prop/direct/P1963'), ('oname', 'properties for this type')]


1

In [44]:
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?monarchyname ?countryname
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Eastern Hungarian Kingdom'), ('countryname', 'Principality of Transylvania')]
[('monarchyname', 'East Francia'), ('countryname', 'Holy Roman Empire')]
[('monarchyname', 'Province of North Carolina'), ('countryname', 'United States of America')]
[('monarchyname', 'Song dynasty'), ('countryname', 'Yuan Empire')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania')]
[('monarchyname', 'United Principalities'), ('countryname', 'Kingdom of Romania')]
[('monarchyname', 'Travancore'), ('countryname', 'India')]
[('monarchyname', 'Travancore'), ('countryname', 'India')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Mataram Sultanate'), ('countryname', 'Indonesia')]
[('monarchyname', 'Kingdom of Cochin'), ('countryname', 'India')]
[('monarchyname', 'Kingdom of C

40

In [45]:
#[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1282'), ('s', 'Key:start_date'), ('oname', 'inception')]
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT DISTINCT ?monarchyname ?countryname ?year
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    BIND(year(?date) AS ?year)
    #FILTER(REGEX(STR(?pname), "^[Cc]ountry"))
} 
ORDER BY DESC(?year)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kingdom of Nepal'), ('countryname', 'Nepal'), ('year', '2008')]
[('monarchyname', 'Dominion of New Zealand'), ('countryname', 'New Zealand'), ('year', '1986')]
[('monarchyname', 'Southern Rhodesia'), ('countryname', 'Zimbabwe'), ('year', '1980')]
[('monarchyname', 'Kingdom of Zimbabwe'), ('countryname', 'Zimbabwe'), ('year', '1980')]
[('monarchyname', 'Pahlavi Dynasty'), ('countryname', 'Iran'), ('year', '1979')]
[('monarchyname', 'British Solomon Islands'), ('countryname', 'Solomon Islands'), ('year', '1978')]
[('monarchyname', 'Kingdom of Libya'), ('countryname', 'Libya'), ('year', '1977')]
[('monarchyname', 'Đại Việt'), ('countryname', 'Vietnam'), ('year', '1976')]
[('monarchyname', 'Dominion of Ceylon'), ('countryname', 'Sri Lanka'), ('year', '1972')]
[('monarchyname', 'Kingdom of Libya'), ('countryname', 'Libya'), ('year', '1969')]
[('monarchyname', 'State of Malta'), ('countryname', 'Malta'), ('year', '1964')]
[('monarchyname', 'July Monarchy'), ('count

40

## Show the last monarchies in the world.

In [46]:
#[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1282'), ('s', 'Key:start_date'), ('oname', 'inception')]
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT DISTINCT ?inception ?monarchyname ?countryname 
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    BIND(year(?date) AS ?inception)

} 
ORDER BY DESC(?inception)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('inception', '2008'), ('monarchyname', 'Kingdom of Nepal'), ('countryname', 'Nepal')]
[('inception', '1986'), ('monarchyname', 'Dominion of New Zealand'), ('countryname', 'New Zealand')]
[('inception', '1980'), ('monarchyname', 'Southern Rhodesia'), ('countryname', 'Zimbabwe')]
[('inception', '1980'), ('monarchyname', 'Kingdom of Zimbabwe'), ('countryname', 'Zimbabwe')]
[('inception', '1979'), ('monarchyname', 'Pahlavi Dynasty'), ('countryname', 'Iran')]
[('inception', '1978'), ('monarchyname', 'British Solomon Islands'), ('countryname', 'Solomon Islands')]
[('inception', '1977'), ('monarchyname', 'Kingdom of Libya'), ('countryname', 'Libya')]
[('inception', '1976'), ('monarchyname', 'Đại Việt'), ('countryname', 'Vietnam')]
[('inception', '1972'), ('monarchyname', 'Dominion of Ceylon'), ('countryname', 'Sri Lanka')]
[('inception', '1969'), ('monarchyname', 'Kingdom of Libya'), ('countryname', 'Libya')]
[('inception', '1964'), ('monarchyname', 'State of Malta'), ('countryname'

40

In [47]:
#[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1282'), ('s', 'Key:start_date'), ('oname', 'inception')]
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT DISTINCT ?inception ?monarchyname ?replacedbyname ?countryname 
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    BIND(year(?date) AS ?inception)

} 
ORDER BY DESC(?inception)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('inception', '2008'), ('monarchyname', 'Kingdom of Nepal'), ('replacedbyname', 'Nepal'), ('countryname', 'Nepal')]
[('inception', '1986'), ('monarchyname', 'Dominion of New Zealand'), ('replacedbyname', 'New Zealand'), ('countryname', 'New Zealand')]
[('inception', '1980'), ('monarchyname', 'Southern Rhodesia'), ('replacedbyname', 'Zimbabwe'), ('countryname', 'Zimbabwe')]
[('inception', '1980'), ('monarchyname', 'Kingdom of Zimbabwe'), ('replacedbyname', 'Kingdom of Butua'), ('countryname', 'Zimbabwe')]
[('inception', '1979'), ('monarchyname', 'Pahlavi Dynasty'), ('replacedbyname', 'Iran'), ('countryname', 'Iran')]
[('inception', '1978'), ('monarchyname', 'British Solomon Islands'), ('replacedbyname', 'Solomon Islands'), ('countryname', 'Solomon Islands')]
[('inception', '1977'), ('monarchyname', 'Kingdom of Libya'), ('replacedbyname', 'Libya'), ('countryname', 'Libya')]
[('inception', '1976'), ('monarchyname', 'Đại Việt'), ('replacedbyname', 'Vietnam'), ('countryname', 'Viet

40

## Count the number of monarchies every country had and show which country had more monarchies in history. 

In [48]:
#[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1282'), ('s', 'Key:start_date'), ('oname', 'inception')]
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?countryname (COUNT(DISTINCT ?monarchy) AS ?countmonarchies)
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    BIND(year(?date) AS ?inception)

} 
ORDER BY DESC(?countmonarchies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countryname', 'India'), ('countmonarchies', '20')]
[('countryname', 'Spain'), ('countmonarchies', '9')]
[('countryname', 'France'), ('countmonarchies', '8')]
[('countryname', 'German Empire'), ('countmonarchies', '5')]
[('countryname', 'Ottoman Empire'), ('countmonarchies', '5')]
[('countryname', 'Kingdom of Italy'), ('countmonarchies', '5')]
[('countryname', 'Japan'), ('countmonarchies', '5')]
[('countryname', 'Weimar Republic'), ('countmonarchies', '4')]
[('countryname', 'Kingdom of Prussia'), ('countmonarchies', '4')]
[('countryname', 'United States of America'), ('countmonarchies', '4')]


10

In [49]:
#[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1282'), ('s', 'Key:start_date'), ('oname', 'inception')]
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?countryname (COUNT(DISTINCT ?monarchy) AS ?countmonarchies)
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
    ?country <http://schema.org/name> ?countryname .


} 
ORDER BY DESC(?countmonarchies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countryname', 'India'), ('countmonarchies', '49')]
[('countryname', 'Spain'), ('countmonarchies', '13')]
[('countryname', 'France'), ('countmonarchies', '11')]
[('countryname', 'German Empire'), ('countmonarchies', '9')]
[('countryname', 'Weimar Republic'), ('countmonarchies', '8')]
[('countryname', 'Ottoman Empire'), ('countmonarchies', '7')]
[('countryname', 'Nazi Germany'), ('countmonarchies', '7')]
[('countryname', 'South Africa'), ('countmonarchies', '6')]
[('countryname', 'Kingdom of Prussia'), ('countmonarchies', '5')]
[('countryname', 'Kingdom of Italy'), ('countmonarchies', '5')]


10

In [50]:
#[('o', 'http://www.wikidata.org/entity/P571'), ('p', 'http://www.wikidata.org/prop/direct/P1282'), ('s', 'Key:start_date'), ('oname', 'inception')]
# COUNTRY 
#[('p', 'http://www.wikidata.org/entity/Q189326'), ('o', 'http://www.wikidata.org/prop/direct/P1366'), ('oname', 'replaced by')]
queryString = """
SELECT ?countryname (COUNT(DISTINCT ?monarchy) AS ?countmonarchies)
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replacedby <http://schema.org/name> ?replacedbyname .
    ?country <http://schema.org/name> ?countryname .


} 
ORDER BY DESC(?countmonarchies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countryname', 'India'), ('countmonarchies', '49')]
[('countryname', 'Spain'), ('countmonarchies', '14')]
[('countryname', 'France'), ('countmonarchies', '11')]
[('countryname', 'German Empire'), ('countmonarchies', '9')]
[('countryname', 'Weimar Republic'), ('countmonarchies', '8')]
[('countryname', 'Ottoman Empire'), ('countmonarchies', '7')]
[('countryname', 'Nazi Germany'), ('countmonarchies', '7')]
[('countryname', 'South Africa'), ('countmonarchies', '6')]
[('countryname', 'Kingdom of Prussia'), ('countmonarchies', '5')]
[('countryname', 'Kingdom of Italy'), ('countmonarchies', '5')]


10

## Identify the BGP for obtaining statistical information of monarchies, e.g., population or inception date

In [51]:
# COUNTRY 

queryString = """
SELECT DISTINCT ?inception ?monarchyname ?replacedbyname ?countryname 
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    BIND(year(?date) AS ?inception)

} 
ORDER BY DESC(?inception)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('inception', '2008'), ('monarchyname', 'Kingdom of Nepal'), ('replacedbyname', 'Nepal'), ('countryname', 'Nepal')]
[('inception', '1986'), ('monarchyname', 'Dominion of New Zealand'), ('replacedbyname', 'New Zealand'), ('countryname', 'New Zealand')]
[('inception', '1980'), ('monarchyname', 'Southern Rhodesia'), ('replacedbyname', 'Zimbabwe'), ('countryname', 'Zimbabwe')]
[('inception', '1980'), ('monarchyname', 'Kingdom of Zimbabwe'), ('replacedbyname', 'Kingdom of Butua'), ('countryname', 'Zimbabwe')]
[('inception', '1979'), ('monarchyname', 'Pahlavi Dynasty'), ('replacedbyname', 'Iran'), ('countryname', 'Iran')]
[('inception', '1978'), ('monarchyname', 'British Solomon Islands'), ('replacedbyname', 'Solomon Islands'), ('countryname', 'Solomon Islands')]
[('inception', '1977'), ('monarchyname', 'Kingdom of Libya'), ('replacedbyname', 'Libya'), ('countryname', 'Libya')]
[('inception', '1976'), ('monarchyname', 'Đại Việt'), ('replacedbyname', 'Vietnam'), ('countryname', 'Viet

10

In [52]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]
queryString = """
SELECT *
WHERE { 

    wd:Q207991 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('o', 'http://www.wikidata.org/entity/Q182921'), ('pname', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'http://www.wikidata.org/entity/Q128135'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'http://www.wikidata.org/entity/Q7269'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0048220'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('o', 'http://www.wikidata.org/entity/Q189326'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('o', 'http://www.wikidata.org/entity/Q794'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('o', 'topic/Pahlavi-dynasty'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1549'), ('o', 'Iranian'), ('pname', 'demonym'

10

In [53]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]
queryString = """
SELECT COUNT(DISTINCT ?p)
WHERE { 

    wd:Q207991 ?p ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '31')]


1

In [54]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]
queryString = """
SELECT DISTINCT ?p ?pname ?o
WHERE { 

    wd:Q207991 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 31
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by'), ('o', 'http://www.wikidata.org/entity/Q182921')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q128135')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q7269')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID'), ('o', '0048220')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q189326')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by'), ('o', 'http://www.wikidata.org/entity/Q794')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID'), ('o', 'topic/Pahlavi-dynasty')]
[('p', 'http://www.wikidata.org/prop/direct/P1549'), ('pname', 'demonym'), ('o', 'Iranian'

30

In [55]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]
queryString = """
SELECT DISTINCT ?p ?pname ?oname
WHERE { 

    wd:Q207991 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 31
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces'), ('oname', 'Qajar dynasty')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by'), ('oname', 'Reza Shah')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('oname', 'dynasty')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('oname', 'historical country')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('pname', 'capital'), ('oname', 'Tehran')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent'), ('oname', 'Asia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('oname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('oname', 'monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by'), ('oname', 'Iran')]
[('p', 'http://www.wikidata.org/prop/direct/P37'), ('pname', 'official language

14

In [56]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]
# [('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
# [('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception')]
# [('p', 'http://www.wikidata.org/prop/direct/P36'), ('pname', 'capital')]
# [('p', 'http://www.wikidata.org/prop/direct/P38'), ('pname', 'currency')]
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q6256')]
## DATE 
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('oname', 'historical country')] then below
# [('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P580'),
#('pname', 'inappropriate property for this type'), ('oname', 'start time')]
#[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P582'), 
#('pname', 'inappropriate property for this type'), ('oname', 'end time')]
#[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '2437574')]

queryString = """
SELECT DISTINCT *
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent .
    
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .

    
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q139315'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('monarchyname', 'Dʿmt'), ('continentname', 'Africa')]
[('monarchy', 'http://www.wikidata.org/entity/Q688859'), ('continent', 'http://www.wikidata.org/entity/Q46'), ('monarchyname', 'Duchy of Bouillon'), ('continentname', 'Europe')]
[('monarchy', 'http://www.wikidata.org/entity/Q2308237'), ('continent', 'http://www.wikidata.org/entity/Q48'), ('monarchyname', 'Ahom Kingdom'), ('continentname', 'Asia')]
[('monarchy', 'http://www.wikidata.org/entity/Q6207845'), ('continent', 'http://www.wikidata.org/entity/Q48'), ('monarchyname', 'Jodhpur State'), ('continentname', 'Asia')]
[('monarchy', 'http://www.wikidata.org/entity/Q1740533'), ('continent', 'http://www.wikidata.org/entity/Q5401'), ('monarchyname', 'Baku Khanate'), ('continentname', 'Eurasia')]
[('monarchy', 'http://www.wikidata.org/entity/Q625380'), ('continent', 'http://www.wikidata.org/entity/Q46'), ('monarchyname', 

10

In [57]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]

# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q6256')]
## DATE 
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('oname', 'historical country')] then below
# [('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P580'),
#('pname', 'inappropriate property for this type'), ('oname', 'start time')]
#[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P582'), 
#('pname', 'inappropriate property for this type'), ('oname', 'end time')]


queryString = """
SELECT DISTINCT *
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
               
               
    
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .


    
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q156418'), ('continent', 'http://www.wikidata.org/entity/Q538'), ('inception', '1795-01-01T00:00:00Z'), ('capital', 'http://www.wikidata.org/entity/Q216258'), ('currency', 'http://www.wikidata.org/entity/Q4917'), ('population', '1217000'), ('monarchyname', 'Kingdom of Hawaiʻi'), ('continentname', 'Insular Oceania'), ('capitalname', 'Hilo'), ('currencyname', 'United States dollar')]
[('monarchy', 'http://www.wikidata.org/entity/Q156418'), ('continent', 'http://www.wikidata.org/entity/Q538'), ('inception', '1795-01-01T00:00:00Z'), ('capital', 'http://www.wikidata.org/entity/Q216258'), ('currency', 'http://www.wikidata.org/entity/Q2328868'), ('population', '1217000'), ('monarchyname', 'Kingdom of Hawaiʻi'), ('continentname', 'Insular Oceania'), ('capitalname', 'Hilo'), ('currencyname', 'Hawaiian dollar')]
[('monarchy', 'http://www.wikidata.org/entity/Q156418'), ('continent', 'http://www.wikidata.org/entity/Q538'), ('inception', '1795-0

10

In [58]:
# [('monarchyname', 'Pahlavi Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q207991'), 
#('countryname', 'Iran'), ('country', 'http://www.wikidata.org/entity/Q794')]

# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q6256')]
## DATE 
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('oname', 'historical country')] then below
# [('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P580'),
#('pname', 'inappropriate property for this type'), ('oname', 'start time')]
#[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('o', 'http://www.wikidata.org/entity/P582'), 
#('pname', 'inappropriate property for this type'), ('oname', 'end time')]


queryString = """
SELECT DISTINCT *
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
    ?monarchy wdt:P31 ?historicalcountry .


               
               
    
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .



    
} 
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q156418'), ('continent', 'http://www.wikidata.org/entity/Q538'), ('inception', '1795-01-01T00:00:00Z'), ('capital', 'http://www.wikidata.org/entity/Q216258'), ('currency', 'http://www.wikidata.org/entity/Q4917'), ('population', '1217000'), ('historicalcountry', 'http://www.wikidata.org/entity/Q3024240'), ('monarchyname', 'Kingdom of Hawaiʻi'), ('continentname', 'Insular Oceania'), ('capitalname', 'Hilo'), ('currencyname', 'United States dollar')]


1

In [59]:
queryString = """
SELECT DISTINCT ?monarchyname ?population ?inception ?currencyname ?capitalname ?capital
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
   # ?monarchy wdt:P1366 ?replacedby .
   # ?replacedby wdt:P17 ?country .
    ?monarchy wdt:P31 ?historicalcountry .
    ?historicalcountry wdt:P17 ?country .

               
               
    
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
    #?country <http://schema.org/name> ?countryname .


    
} 
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kingdom of Dalmatia'), ('population', '645666'), ('inception', '1815-06-22T00:00:00Z'), ('currencyname', 'Austro-Hungarian gulden'), ('capitalname', 'Zadar'), ('capital', 'http://www.wikidata.org/entity/Q3370')]


1

In [60]:
# where is the country that every capital is located?
queryString = """
SELECT *
WHERE { 

    wd:Q3370 wdt:P17 ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q224'), ('oname', 'Croatia')]


1

## Identify the BGP for obtaining statistical information of monarchies, e.g., poluation or inception date

In [61]:
queryString = """
SELECT ?monarchyname ?population ?inception ?currencyname ?capitalname ?capitallocationname
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
    ?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
    ?capitallocation <http://schema.org/name> ?capitallocationname .

} 
ORDER BY ASC(?inception)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Byzantine Empire'), ('population', '17000000'), ('inception', '0395-01-18T00:00:00Z'), ('currencyname', 'Byzantine coinage'), ('capitalname', 'Syracuse'), ('capitallocationname', 'Italy')]
[('monarchyname', 'Byzantine Empire'), ('population', '17000000'), ('inception', '0395-01-18T00:00:00Z'), ('currencyname', 'ducat'), ('capitalname', 'Syracuse'), ('capitallocationname', 'Italy')]
[('monarchyname', 'Byzantine Empire'), ('population', '17000000'), ('inception', '0395-01-18T00:00:00Z'), ('currencyname', 'solidus'), ('capitalname', 'Syracuse'), ('capitallocationname', 'Italy')]
[('monarchyname', 'Byzantine Empire'), ('population', '17000000'), ('inception', '0395-01-18T00:00:00Z'), ('currencyname', 'Byzantine coinage'), ('capitalname', 'Constantinople'), ('capitallocationname', 'Roman Empire')]
[('monarchyname', 'Byzantine Empire'), ('population', '17000000'), ('inception', '0395-01-18T00:00:00Z'), ('currencyname', 'Byzantine coinage'), ('capitalname', 'Constan

10

## How many monarchies have used dollar and euro as a currency. 

In [62]:
queryString = """
SELECT (COUNT(DISTINCT ?monarchyname) AS ?countcurrency)
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
    ?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
#    ?continent <http://schema.org/name> ?continentname .
#    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
#    ?capitallocation <http://schema.org/name> ?capitallocationname .
    FILTER(REGEX(STR(?currencyname), "[Dd][ollar]")) 


} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countcurrency', '27')]


1

In [63]:
queryString = """
SELECT ?monarchyname ?currencyname 
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
    ?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
#    ?continent <http://schema.org/name> ?continentname .
#    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
#    ?capitallocation <http://schema.org/name> ?capitallocationname .
    FILTER(REGEX(STR(?currencyname), "[Dd][ollar]")) 


} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'Hawaiian dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'Hawaiian dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'Hawaiian dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'Hawaiian dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]


10

In [64]:
queryString = """
SELECT (COUNT(?monarchyname) AS ?countcurrency)
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P38 ?currency .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?currency <http://schema.org/name> ?currencyname .
    FILTER regex(?currencyname, "dollar", "i") 


} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countcurrency', '47')]


1

In [65]:
queryString = """
SELECT DISTINCT ?monarchyname ?currencyname
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P38 ?currency .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?currency <http://schema.org/name> ?currencyname .
    FILTER regex(?currencyname, "dollar", "i") 


} 

LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('monarchyname', 'Kingdom of Hawaiʻi'), ('currencyname', 'Hawaiian dollar')]
[('monarchyname', 'Islands of Refreshment'), ('currencyname', 'spanish dollar')]
[('monarchyname', 'Malayan Union'), ('currencyname', 'Malayan dollar')]
[('monarchyname', 'Rhodesia'), ('currencyname', 'Rhodesian dollar')]
[('monarchyname', 'Canada'), ('currencyname', 'Canadian dollar')]
[('monarchyname', 'Belize'), ('currencyname', 'Belize dollar')]
[('monarchyname', 'Dominion of Newfoundland'), ('currencyname', 'Newfoundland dollar')]
[('monarchyname', 'Australia'), ('currencyname', 'Australian dollar')]
[('monarchyname', 'New Zealand'), ('currencyname', 'New Zealand dollar')]
[('monarchyname', 'Saint Vincent and the Grenadines'), ('currencyname', 'Eastern Caribbean dollar')]
[('monarchyname', 'Saint Lucia'), ('currencyname', 'Eastern Caribbean dollar')]
[('monarchyname', 'Saint Kitts and Nevis'), ('currencyname', 'Eas

34

In [66]:
queryString = """
SELECT DISTINCT ?monarchyname ?currencyname ?inception ?capitalname ?capitallocationname
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P38 ?currency ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P1082 ?population .
    ?capital wdt:P17 ?capitallocation .
  
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
    ?capitallocation <http://schema.org/name> ?capitallocationname .

    FILTER regex(?currencyname, "euro", "i") 


} 

LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kingdom of the Netherlands'), ('currencyname', 'euro'), ('inception', '1815-03-16T00:00:00Z'), ('capitalname', 'Amsterdam'), ('capitallocationname', 'Netherlands')]
[('monarchyname', 'Spain'), ('currencyname', 'euro'), ('inception', '1479-01-01T00:00:00Z'), ('capitalname', 'Madrid'), ('capitallocationname', 'Spain')]
[('monarchyname', 'Netherlands'), ('currencyname', 'euro'), ('inception', '1954-12-15T00:00:00Z'), ('capitalname', 'Amsterdam'), ('capitallocationname', 'Kingdom of the Netherlands')]
[('monarchyname', 'Monaco'), ('currencyname', 'euro'), ('inception', '1297-01-15T00:00:00Z'), ('capitalname', 'Monaco City'), ('capitallocationname', 'Monaco')]
[('monarchyname', 'Belgium'), ('currencyname', 'euro'), ('inception', '1830-10-04T00:00:00Z'), ('capitalname', 'City of Brussels'), ('capitallocationname', 'Belgium')]
[('monarchyname', 'Kingdom of the Netherlands'), ('currencyname', 'euro'), ('inception', '1815-03-16T00:00:00Z'), ('capitalname', 'Amsterdam'

9

In [67]:
queryString = """
SELECT DISTINCT ?monarchyname ?monarchy ?currencyname
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P38 ?currency .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?currency <http://schema.org/name> ?currencyname .
    FILTER regex(?currencyname, "euro", "i") 


} 

LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Andorra'), ('monarchy', 'http://www.wikidata.org/entity/Q228'), ('currencyname', 'euro')]
[('monarchyname', 'Monaco'), ('monarchy', 'http://www.wikidata.org/entity/Q235'), ('currencyname', 'euro')]
[('monarchyname', 'Vatican City'), ('monarchy', 'http://www.wikidata.org/entity/Q237'), ('currencyname', 'euro')]
[('monarchyname', 'Spain'), ('monarchy', 'http://www.wikidata.org/entity/Q29'), ('currencyname', 'euro')]
[('monarchyname', 'Kingdom of the Netherlands'), ('monarchy', 'http://www.wikidata.org/entity/Q29999'), ('currencyname', 'euro')]
[('monarchyname', 'Belgium'), ('monarchy', 'http://www.wikidata.org/entity/Q31'), ('currencyname', 'euro')]
[('monarchyname', 'Luxembourg'), ('monarchy', 'http://www.wikidata.org/entity/Q32'), ('currencyname', 'euro')]
[('monarchyname', 'Netherlands'), ('monarchy', 'http://www.wikidata.org/entity/Q55'), ('currencyname', 'euro')]
[('monarchyname', 'Gay and Lesbian Kingdom of the Coral Sea Islands'), ('monarchy', 'http://ww

9

In [68]:
queryString = """
SELECT *
WHERE { 

    wd:Q228 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "loc", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1740'), ('o', 'http://www.wikidata.org/entity/Q19363033'), ('pname', 'category for films shot at this location')]
[('p', 'http://www.wikidata.org/prop/direct/P1943'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Andorra%20location%20map.svg'), ('pname', 'location map')]
[('p', 'http://www.wikidata.org/prop/direct/P1944'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Andorra%20relief%20location%20map.jpg'), ('pname', 'relief location map')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Andorra%20on%20the%20globe%20%28Europe%20centered%29.svg'), ('pname', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Location%20Andorra%20Europe.png'), ('pname', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('o', 'http://commons.wikimedia.org/wiki/Speci

14

## Which is or was  the country with the highest population?

In [69]:
queryString = """
SELECT (MAX(?population) AS ?maxpopulation)
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
    ?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
    ?capitallocation <http://schema.org/name> ?capitallocationname .

} 

"""

print("Results")
run_query(queryString)

Results
[('maxpopulation', '680000000')]


1

In [70]:
queryString = """
SELECT ?monarchyname ?population ?capitallocation
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent ;
               wdt:P571 ?inception ;
               wdt:P36 ?capital ;
               wdt:P38 ?currency ;
               wdt:P1082 ?population .
    ?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .
    ?capital <http://schema.org/name> ?capitalname .
    ?currency <http://schema.org/name> ?currencyname .
    ?capitallocation <http://schema.org/name> ?capitallocationname .

} 
ORDER BY DESC (?population)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'British Empire'), ('population', '680000000'), ('capitallocation', 'http://www.wikidata.org/entity/Q145')]


1

## How many countries are or have been monarchies across continents?

In [71]:
queryString = """
SELECT ?continentname (COUNT(?monarchy) AS ?monarchycount)
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P30 ?continent .

    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?continent <http://schema.org/name> ?continentname .


} 
GROUP BY ?continentname
ORDER BY DESC (?monarchycount)
"""

print("Results")
run_query(queryString)

Results
[('continentname', 'Europe'), ('monarchycount', '478')]
[('continentname', 'Asia'), ('monarchycount', '207')]
[('continentname', 'Africa'), ('monarchycount', '72')]
[('continentname', 'North America'), ('monarchycount', '41')]
[('continentname', 'Insular Oceania'), ('monarchycount', '20')]
[('continentname', 'Eurasia'), ('monarchycount', '16')]
[('continentname', 'Hyboria'), ('monarchycount', '13')]
[('continentname', 'South America'), ('monarchycount', '8')]
[('continentname', 'The Continent'), ('monarchycount', '7')]
[('continentname', 'Americas'), ('monarchycount', '7')]
[('continentname', 'Thuria'), ('monarchycount', '6')]
[('continentname', 'Central America'), ('monarchycount', '2')]
[('continentname', 'Australia'), ('monarchycount', '2')]
[('continentname', 'Northern America and Mexico'), ('monarchycount', '1')]
[('continentname', 'Antarctica'), ('monarchycount', '1')]
[('continentname', 'Caribbean'), ('monarchycount', '1')]


16

## Analyze the number of countries that were monarchies

### How many sub-types of monarchy exist?  

In [72]:
# the number of sub types.
queryString = """
SELECT COUNT(DISTINCT ?subtypes) AS ?countsubtype
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .

} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countsubtype', '24')]


1

In [73]:
queryString = """
SELECT ?subtypes ?subtypesname
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .

    # Labels

    ?subtypes <http://schema.org/name> ?subtypesname .

} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('subtypes', 'http://www.wikidata.org/entity/Q3772929'), ('subtypesname', 'Dewan')]
[('subtypes', 'http://www.wikidata.org/entity/Q188800'), ('subtypesname', 'personal union')]
[('subtypes', 'http://www.wikidata.org/entity/Q521263'), ('subtypesname', 'history of Tibet')]
[('subtypes', 'http://www.wikidata.org/entity/Q830852'), ('subtypesname', 'history of ancient Rome')]
[('subtypes', 'http://www.wikidata.org/entity/Q463573'), ('subtypesname', 'suzerainty')]
[('subtypes', 'http://www.wikidata.org/entity/Q166886'), ('subtypesname', 'duke')]
[('subtypes', 'http://www.wikidata.org/entity/Q3024240'), ('subtypesname', 'historical country')]
[('subtypes', 'http://www.wikidata.org/entity/Q3024240'), ('subtypesname', 'historical country')]
[('subtypes', 'http://www.wikidata.org/entity/Q43702'), ('subtypesname', 'federation')]
[('subtypes', 'http://www.wikidata.org/entity/Q48349'), ('subtypesname', 'empire')]


10

In [74]:
queryString = """
SELECT ?subtypes ?subtypesname
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?subtypes  wdt:P279 ?monarchy .

    # Labels

    ?subtypes <http://schema.org/name> ?subtypesname .

} 
ORDER BY ASC(?inception)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('subtypes', 'http://www.wikidata.org/entity/Q27925057'), ('subtypesname', 'Electoral Principate')]
[('subtypes', 'http://www.wikidata.org/entity/Q188800'), ('subtypesname', 'personal union')]
[('subtypes', 'http://www.wikidata.org/entity/Q3403564'), ('subtypesname', 'ecclesiastical principality')]
[('subtypes', 'http://www.wikidata.org/entity/Q48349'), ('subtypesname', 'empire')]
[('subtypes', 'http://www.wikidata.org/entity/Q417175'), ('subtypesname', 'kingdom')]
[('subtypes', 'http://www.wikidata.org/entity/Q2494447'), ('subtypesname', 'part principality')]
[('subtypes', 'http://www.wikidata.org/entity/Q6025390'), ('subtypesname', 'Labour movement in Spain')]
[('subtypes', 'http://www.wikidata.org/entity/Q389688'), ('subtypesname', 'Achaemenid Empire')]
[('subtypes', 'http://www.wikidata.org/entity/Q12759805'), ('subtypesname', 'sultanate')]
[('subtypes', 'http://www.wikidata.org/entity/Q165116'), ('subtypesname', 'grand duchy')]


10

### How many instances of exist of each one?

In [75]:
# the number of instance.
queryString = """
SELECT COUNT(DISTINCT ?instance) AS ?countinstance
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
    
} 

"""

print("Results")
run_query(queryString)

Results
[('countinstance', '14')]


1

In [76]:
# the number of instance.
queryString = """
SELECT COUNT(?instance) AS ?countinstance
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .

} 

"""

print("Results")
run_query(queryString)

Results
[('countinstance', '26')]


1

In [77]:
# the number of instance.
queryString = """
SELECT *
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
        # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?subtypes <http://schema.org/name> ?subtypesname .
    ?instance <http://schema.org/name> ?instancename .
} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q1250464'), ('subtypes', 'http://www.wikidata.org/entity/Q183366'), ('instance', 'http://www.wikidata.org/entity/Q15617994'), ('monarchyname', 'realm'), ('subtypesname', 'territory'), ('instancename', 'designation for an administrative territorial entity')]
[('monarchy', 'http://www.wikidata.org/entity/Q1250464'), ('subtypes', 'http://www.wikidata.org/entity/Q7275'), ('instance', 'http://www.wikidata.org/entity/Q106589826'), ('monarchyname', 'realm'), ('subtypesname', 'state'), ('instancename', 'Finnish National Land Survey place type')]
[('monarchy', 'http://www.wikidata.org/entity/Q126416'), ('subtypes', 'http://www.wikidata.org/entity/Q521263'), ('instance', 'http://www.wikidata.org/entity/Q17544377'), ('monarchyname', 'Phagmodrupa Dynasty'), ('subtypesname', 'history of Tibet'), ('instancename', 'history of a country or state')]
[('monarchy', 'http://www.wikidata.org/entity/Q725045'), ('subtypes', 'http://www.wikidata.org/entity

25

In [78]:
# show the instance name.
queryString = """
SELECT ?monarchyname ?subtypesname 
(GROUP_CONCAT(DISTINCT ?instancename; SEPARATOR=", ") AS ?instancenames) 
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
        # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?subtypes <http://schema.org/name> ?subtypesname .
    ?instance <http://schema.org/name> ?instancename .
} 

LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'First Toungoo Empire'), ('subtypesname', 'suzerainty'), ('instancenames', 'form of state')]
[('monarchyname', 'Pre-modern Hungary'), ('subtypesname', 'nobility of Hungary'), ('instancenames', 'social class')]
[('monarchyname', 'Modern Saudi State'), ('subtypesname', 'Modern history of Saudi Arabia'), ('instancenames', 'aspect of history')]
[('monarchyname', 'Burmese kingdoms'), ('subtypesname', 'history of monarchy'), ('instancenames', 'aspect of history')]
[('monarchyname', 'Duchy of Brittany'), ('subtypesname', 'duchy'), ('instancenames', 'designation for an administrative territorial entity')]
[('monarchyname', 'principality'), ('subtypesname', 'monarchy'), ('instancenames', 'form of government')]
[('monarchyname', 'Kindah Kingdom'), ('subtypesname', 'History of the Arabian Kingdom by period\u200e'), ('instancenames', 'history by period')]
[('monarchyname', 'grand duchy'), ('subtypesname', 'principality'), ('instancenames', 'designation for an administrati

19

In [79]:
# 
queryString = """
SELECT ?monarchyname ?monarchy ?subtypesname ?instancename
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
        # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?subtypes <http://schema.org/name> ?subtypesname .
    ?instance <http://schema.org/name> ?instancename .
} 
GROUP BY ?subtypesname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'grand duchy'), ('monarchy', 'http://www.wikidata.org/entity/Q165116'), ('subtypesname', 'principality'), ('instancename', 'designation for an administrative territorial entity')]
[('monarchyname', 'Phagmodrupa Dynasty'), ('monarchy', 'http://www.wikidata.org/entity/Q126416'), ('subtypesname', 'history of Tibet'), ('instancename', 'history of a country or state')]
[('monarchyname', 'Pflegamt'), ('monarchy', 'http://www.wikidata.org/entity/Q22693483'), ('subtypesname', 'state'), ('instancename', 'Finnish National Land Survey place type')]
[('monarchyname', 'Pre-modern Hungary'), ('monarchy', 'http://www.wikidata.org/entity/Q89101318'), ('subtypesname', 'nobility of Hungary'), ('instancename', 'social class')]
[('monarchyname', 'Duchess of Palma de Mallorca'), ('monarchy', 'http://www.wikidata.org/entity/Q3323826'), ('subtypesname', 'substantive title'), ('instancename', 'Wikimedia list article')]
[('monarchyname', 'principality'), ('monarchy', 'http://www.wikid

20

In [80]:
queryString = """
SELECT *
WHERE { 
    #?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    wd:Q389688 wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
        # Labels

    #?monarchy <http://schema.org/name> ?monarchyname .
    ?subtypes <http://schema.org/name> ?subtypesname .
    ?instance <http://schema.org/name> ?instancename .
} 

LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('subtypes', 'http://www.wikidata.org/entity/Q83311'), ('instance', 'http://www.wikidata.org/entity/Q11514315'), ('subtypesname', 'Persian Empire'), ('instancename', 'historical period')]
[('subtypes', 'http://www.wikidata.org/entity/Q83311'), ('instance', 'http://www.wikidata.org/entity/Q2221906'), ('subtypesname', 'Persian Empire'), ('instancename', 'geographic location')]
[('subtypes', 'http://www.wikidata.org/entity/Q83311'), ('instance', 'http://www.wikidata.org/entity/Q7269'), ('subtypesname', 'Persian Empire'), ('instancename', 'monarchy')]


3

In [81]:
queryString = """
SELECT ?instance ?instancename
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
        # Labels

    ?instance <http://schema.org/name> ?instancename .
} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q11514315'), ('instancename', 'historical period')]
[('instance', 'http://www.wikidata.org/entity/Q1307214'), ('instancename', 'form of government')]
[('instance', 'http://www.wikidata.org/entity/Q1307214'), ('instancename', 'form of government')]
[('instance', 'http://www.wikidata.org/entity/Q1307214'), ('instancename', 'form of government')]
[('instance', 'http://www.wikidata.org/entity/Q1307214'), ('instancename', 'form of government')]
[('instance', 'http://www.wikidata.org/entity/Q13406463'), ('instancename', 'Wikimedia list article')]
[('instance', 'http://www.wikidata.org/entity/Q17524420'), ('instancename', 'aspect of history')]
[('instance', 'http://www.wikidata.org/entity/Q17524420'), ('instancename', 'aspect of history')]
[('instance', 'http://www.wikidata.org/entity/Q183039'), ('instancename', 'form of state')]
[('instance', 'http://www.wikidata.org/entity/Q183039'), ('instancename', 'form of state')]
[('instance', 'http

26

In [82]:
queryString = """
SELECT DISTINCT ?instance ?instancename
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P279 ?subtypes .
    ?subtypes wdt:P31 ?instance .
        # Labels

    ?instance <http://schema.org/name> ?instancename .
} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q11514315'), ('instancename', 'historical period')]
[('instance', 'http://www.wikidata.org/entity/Q1307214'), ('instancename', 'form of government')]
[('instance', 'http://www.wikidata.org/entity/Q13406463'), ('instancename', 'Wikimedia list article')]
[('instance', 'http://www.wikidata.org/entity/Q17524420'), ('instancename', 'aspect of history')]
[('instance', 'http://www.wikidata.org/entity/Q183039'), ('instancename', 'form of state')]
[('instance', 'http://www.wikidata.org/entity/Q2221906'), ('instancename', 'geographic location')]
[('instance', 'http://www.wikidata.org/entity/Q4164871'), ('instancename', 'position')]
[('instance', 'http://www.wikidata.org/entity/Q7269'), ('instancename', 'monarchy')]
[('instance', 'http://www.wikidata.org/entity/Q15617994'), ('instancename', 'designation for an administrative territorial entity')]
[('instance', 'http://www.wikidata.org/entity/Q17544377'), ('instancename', 'history of a country 

14

### Are there cities that have been capital of multiple kingdoms?

In [83]:
queryString = """
SELECT ?capitalname 
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P36 ?capital .
    #?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?capital <http://schema.org/name> ?capitalname .
    #?capitallocation <http://schema.org/name> ?capitallocationname .

} 
GROUP BY ?capitalname 
HAVING (COUNT(?monarchyname) > 1)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('capitalname', 'Gotha')]
[('capitalname', 'Honolulu')]
[('capitalname', 'Kailua')]
[('capitalname', 'Kavarna')]
[('capitalname', 'Kartasura')]
[('capitalname', 'Stuttgart')]
[('capitalname', 'Kuala Lumpur')]
[('capitalname', 'Carthage')]
[('capitalname', 'Tirana')]
[('capitalname', 'Lobamba')]


10

In [84]:
queryString = """
SELECT ?capitalname (COUNT(DISTINCT ?monarchyname) AS ?countMonarchies) 
WHERE { 
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269 . 
    ?monarchy wdt:P36 ?capital .
    #?capital wdt:P17 ?capitallocation .
  
    # Labels

    ?monarchy <http://schema.org/name> ?monarchyname .
    ?capital <http://schema.org/name> ?capitalname .
    #?capitallocation <http://schema.org/name> ?capitallocationname .

} 
#GROUP BY ?capitalname
#HAVING (?countMonarchies > 1)
ORDER BY DESC(?countMonarchies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('capitalname', 'London'), ('countMonarchies', '8')]
[('capitalname', 'Madrid'), ('countMonarchies', '7')]
[('capitalname', 'Rio de Janeiro'), ('countMonarchies', '7')]
[('capitalname', 'Vienna'), ('countMonarchies', '6')]
[('capitalname', 'Cairo'), ('countMonarchies', '6')]
[('capitalname', 'Paris'), ('countMonarchies', '5')]
[('capitalname', 'Belgrade'), ('countMonarchies', '5')]
[('capitalname', 'Riyadh'), ('countMonarchies', '5')]
[('capitalname', 'Berlin'), ('countMonarchies', '5')]
[('capitalname', 'Bago'), ('countMonarchies', '4')]


10

### Which are the top-3 monarchies with the earliest inception and still existing?

In [85]:
queryString = """
SELECT DISTINCT ?inception ?monarchyname ?monarchy ?replacedbyname ?countryname 
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P1365 ?replaces .
    ?monarchy wdt:P1366 ?replacedby .
    ?replacedby wdt:P17 ?country .
    ?country wdt:P571 ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?replaces <http://schema.org/name> ?replacesname .
    ?replacedby <http://schema.org/name> ?replacedbyname .
        # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    BIND(year(?date) AS ?inception)

} 
ORDER BY ASC(?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('inception', '-1045'), ('monarchyname', 'Chu'), ('monarchy', 'http://www.wikidata.org/entity/Q504759'), ('replacedbyname', 'Qin'), ('countryname', 'Zhou dynasty')]
[('inception', '-1045'), ('monarchyname', 'Yue'), ('monarchy', 'http://www.wikidata.org/entity/Q1137403'), ('replacedbyname', 'Qin'), ('countryname', 'Zhou dynasty')]
[('inception', '-1045'), ('monarchyname', 'Eastern Zhou'), ('monarchy', 'http://www.wikidata.org/entity/Q307066'), ('replacedbyname', 'Qin'), ('countryname', 'Zhou dynasty')]


3

In [86]:
queryString = """
SELECT *
WHERE { 

    wd:Q1137403 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'http://www.wikidata.org/entity/Q7269'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('o', 'http://www.wikidata.org/entity/Q912068'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('o', 'http://www.wikidata.org/entity/Q1437302'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('o', 'http://www.wikidata.org/entity/Q34756'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('o', 'http://www.wikidata.org/entity/Q1074275'), ('pname', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('o', 'place/Yue-ancient-state-China'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q836688'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/en

15

In [87]:
#[('p', 'http://www.wikidata.org/prop/direct/P571'), ('o', '-1100-01-01T00:00:00Z'), ('pname', 'inception')]
#[('p', 'http://www.wikidata.org/prop/direct/P576'), ('o', '-221-01-01T00:00:00Z'), ('pname', 'dissolved, abolished or demolished date')]
queryString = """
SELECT *
WHERE { 

    wd:Q1137403 wdt:P571 ?inception .
    wd:Q1137403 wdt:P576 ?demolished_date .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('inception', '-1100-01-01T00:00:00Z'), ('demolished_date', '-221-01-01T00:00:00Z')]


1

In [88]:
queryString = """
SELECT ?monarchyname ?demolished_date ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    #?monarchy wdt:P1365 ?replaces .
    #?monarchy wdt:P1366 ?replacedby .
    ?monarchy wdt:P571 ?inception .
    ?monarchy wdt:P576 ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #FILTER NOT EXISTS { ?monarchy wdt:P576 ?demolished_date }

} 

LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Dʿmt'), ('demolished_date', '-399-01-01T00:00:00Z'), ('inception', '-979-01-01T00:00:00Z')]
[('monarchyname', 'Dʿmt'), ('demolished_date', '-399-01-01T00:00:00Z'), ('inception', '-979-01-01T00:00:00Z')]
[('monarchyname', 'Ahom Kingdom'), ('demolished_date', '1826-01-01T00:00:00Z'), ('inception', '1228-01-01T00:00:00Z')]


3

In [89]:
queryString = """
SELECT ?monarchyname ?demolished_date ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P571 ?inception .
    ?monarchy wdt:P576 ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .


} 
ORDER BY ASC(?demolished_date)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Western Han'), ('demolished_date', '0009-01-01T00:00:00Z'), ('inception', '-201-01-01T00:00:00Z')]
[('monarchyname', 'Kingdom of Cappadocia'), ('demolished_date', '0017-01-01T00:00:00Z'), ('inception', '-320-01-01T00:00:00Z')]
[('monarchyname', 'Kingdom of Cappadocia'), ('demolished_date', '0017-01-01T00:00:00Z'), ('inception', '-320-01-01T00:00:00Z')]


3

In [90]:
queryString = """
SELECT ?monarchyname ?monarchy ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P571 ?inception .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    FILTER NOT EXISTS { ?monarchy wdt:P576 ?demolished_date }
} 
ORDER BY (?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Galactic Empire'), ('monarchy', 'http://www.wikidata.org/entity/Q2349630'), ('inception', '11584-01-01T00:00:00Z')]
[('monarchyname', 'Roman Catholic Diocese of Metz'), ('monarchy', 'http://www.wikidata.org/entity/Q787336'), ('inception', '0201-01-01T00:00:00Z')]
[('monarchyname', 'Western Roman Empire (395-476 AD)'), ('monarchy', 'http://www.wikidata.org/entity/Q42834'), ('inception', '0395-01-18T00:00:00Z')]


3

In [91]:
queryString = """
SELECT ?monarchyname ?yearstr 
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    #?monarchy wdt:P1365 ?replaces .
    #?monarchy wdt:P1366 ?replacedby .
    ?monarchy wdt:P571 ?inception .
    ?monarchy wdt:P576 ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    
    BIND( year(?demolished_date) as ?year ).
    BIND( str(?year) AS ?yearstr ).
    
} 

ORDER BY (?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kushan Empire'), ('yearstr', '375')]
[('monarchyname', 'Kingdom of Funan'), ('yearstr', '550')]
[('monarchyname', 'Chen princedom'), ('yearstr', '197')]


3

In [92]:
queryString = """
SELECT ?monarchyname
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    #?monarchy wdt:P1365 ?replaces .
    #?monarchy wdt:P1366 ?replacedby .
    ?monarchy wdt:P571 ?inception .
    ?monarchy ?p ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?p <http://schema.org/name> ?pname .
    FILTER(!REGEX(STR(?pname), "demolished"))
} 
ORDER BY (?inception)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Galactic Empire')]
[('monarchyname', 'Kingdom of Araba')]
[('monarchyname', 'Kingdom of Araba')]
[('monarchyname', 'Kingdom of Araba')]


15

In [93]:
queryString = """
SELECT DISTINCT ?monarchyname ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    #?monarchy wdt:P1365 ?replaces .
    #?monarchy wdt:P1366 ?replacedby .
    ?monarchy wdt:P571 ?inception .
    ?monarchy ?p ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?p <http://schema.org/name> ?pname .
    FILTER CONTAINS(?pname, "demolished") .
} 
ORDER BY (?inception)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Kushan Empire'), ('inception', '0030-01-01T00:00:00Z')]
[('monarchyname', 'Kingdom of Funan'), ('inception', '0068-01-01T00:00:00Z')]
[('monarchyname', 'Chen princedom'), ('inception', '0088-01-01T00:00:00Z')]
[('monarchyname', 'Bharshiva Dynasty'), ('inception', '0170-01-01T00:00:00Z')]
[('monarchyname', 'Champa'), ('inception', '0192-01-01T00:00:00Z')]
[('monarchyname', 'Kingdom of Araba'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Shule Kingdom'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Early history of Kedah'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Langkasuka'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Ghassanids'), ('inception', '0220-01-01T00:00:00Z')]
[('monarchyname', 'Shu Han'), ('inception', '0221-01-01T00:00:00Z')]
[('monarchyname', 'Sasanian Empire'), ('inception', '0226-01-01T00:00:00Z')]
[('monarchyname', 'Afrighids'), ('inception', '0305-01-01T00:00:00Z')]
[('monarchyname', 

15

In [94]:
queryString = """
SELECT DISTINCT ?monarchyname ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    #?monarchy wdt:P1365 ?replaces .
    #?monarchy wdt:P1366 ?replacedby .
    ?monarchy wdt:P571 ?inception .
    #?monarchy ?p ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?p <http://schema.org/name> ?pname .
    #FILTER CONTAINS(?pname, "demolished") .
    FILTER NOT EXISTS {?monarchy wdt:P576 ?demolished_date}
} 
ORDER BY (?inception)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Galactic Empire'), ('inception', '11584-01-01T00:00:00Z')]
[('monarchyname', 'Roman Catholic Diocese of Metz'), ('inception', '0201-01-01T00:00:00Z')]
[('monarchyname', 'Western Roman Empire (395-476 AD)'), ('inception', '0395-01-18T00:00:00Z')]
[('monarchyname', 'duchy of Asti'), ('inception', '0569-01-01T00:00:00Z')]
[('monarchyname', 'Roman Catholic Diocese of Bolzano-Brixen'), ('inception', '0600-01-01T00:00:00Z')]
[('monarchyname', 'Sweden'), ('inception', '0700-01-01T00:00:00Z')]
[('monarchyname', 'Danish Realm'), ('inception', '0800-01-01T00:00:00Z')]
[('monarchyname', 'Denmark'), ('inception', '0800-01-01T00:00:00Z')]
[('monarchyname', 'Gansu Uyghur Kingdom'), ('inception', '0848-01-01T00:00:00Z')]
[('monarchyname', 'Norway'), ('inception', '0900-01-01T00:00:00Z')]
[('monarchyname', 'The Moravian Principality'), ('inception', '0900-01-01T00:00:00Z')]
[('monarchyname', 'Principality of Goroden'), ('inception', '1116-01-01T00:00:00Z')]
[('monarchyname',

15

In [95]:
queryString = """
SELECT DISTINCT ?monarchyname ?monarchy ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    #?monarchy wdt:P1365 ?replaces .
    #?monarchy wdt:P1366 ?replacedby .
    ?monarchy wdt:P571 ?inception .
    #?monarchy ?p ?demolished_date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    #?p <http://schema.org/name> ?pname .
    #FILTER CONTAINS(?pname, "demolished") .
    FILTER NOT EXISTS {?monarchy wdt:P576 ?demolished_date}
} 
ORDER BY (?inception)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Galactic Empire'), ('monarchy', 'http://www.wikidata.org/entity/Q2349630'), ('inception', '11584-01-01T00:00:00Z')]
[('monarchyname', 'Roman Catholic Diocese of Metz'), ('monarchy', 'http://www.wikidata.org/entity/Q787336'), ('inception', '0201-01-01T00:00:00Z')]
[('monarchyname', 'Western Roman Empire (395-476 AD)'), ('monarchy', 'http://www.wikidata.org/entity/Q42834'), ('inception', '0395-01-18T00:00:00Z')]
[('monarchyname', 'duchy of Asti'), ('monarchy', 'http://www.wikidata.org/entity/Q384237'), ('inception', '0569-01-01T00:00:00Z')]
[('monarchyname', 'Roman Catholic Diocese of Bolzano-Brixen'), ('monarchy', 'http://www.wikidata.org/entity/Q1249360'), ('inception', '0600-01-01T00:00:00Z')]
[('monarchyname', 'Sweden'), ('monarchy', 'http://www.wikidata.org/entity/Q34'), ('inception', '0700-01-01T00:00:00Z')]
[('monarchyname', 'Danish Realm'), ('monarchy', 'http://www.wikidata.org/entity/Q756617'), ('inception', '0800-01-01T00:00:00Z')]
[('monarchyname', '

15

In [96]:
# Q2349630 'Galactic Empire'
queryString = """
SELECT COUNT(DISTINCT ?p)
WHERE { 

    wd:Q2349630 ?p ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .



    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '15')]


1

In [97]:
# Q2349630 'Galactic Empire'
queryString = """
SELECT DISTINCT ?p ?o ?oname
WHERE { 

    wd:Q2349630 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .



    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('o', 'http://www.wikidata.org/entity/Q52347'), ('oname', 'Galactic Empire')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'http://www.wikidata.org/entity/Q7269'), ('oname', 'monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P1080'), ('o', 'http://www.wikidata.org/entity/Q3551232'), ('oname', 'Foundation Universe')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('o', 'http://www.wikidata.org/entity/Q34981'), ('oname', 'Isaac Asimov')]
[('p', 'http://www.wikidata.org/prop/direct/P35'), ('o', 'http://www.wikidata.org/entity/Q1084312'), ('oname', 'Cleon I')]
[('p', 'http://www.wikidata.org/prop/direct/P35'), ('o', 'http://www.wikidata.org/entity/Q47465486'), ('oname', 'Stanel VI')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('o', 'http://www.wikidata.org/entity/Q52208'), ('oname', 'Trantor')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q52208'), ('ona

20

In [98]:
#[('p', 'http://www.wikidata.org/prop/direct/P31'),
# [('o', 'http://www.wikidata.org/entity/Q19595246'), ('oname', 'fictional empire')]
queryString = """
SELECT *
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P571 ?inception .

    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    FILTER NOT EXISTS {?monarchy wdt:P576 ?demolished_date}
    FILTER NOT EXISTS {?monarchy wdt:P31 wd:Q19595246} 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q1249360'), ('inception', '0600-01-01T00:00:00Z'), ('monarchyname', 'Roman Catholic Diocese of Bolzano-Brixen')]
[('monarchy', 'http://www.wikidata.org/entity/Q16543832'), ('inception', '1140-01-01T00:00:00Z'), ('monarchyname', 'County of Tyrol')]
[('monarchy', 'http://www.wikidata.org/entity/Q1853386'), ('inception', '1757-01-01T00:00:00Z'), ('monarchyname', 'Mangkunegaran')]
[('monarchy', 'http://www.wikidata.org/entity/Q1853386'), ('inception', '1757-01-01T00:00:00Z'), ('monarchyname', 'Mangkunegaran')]
[('monarchy', 'http://www.wikidata.org/entity/Q223'), ('inception', '1979-01-01T00:00:00Z'), ('monarchyname', 'Greenland')]
[('monarchy', 'http://www.wikidata.org/entity/Q756617'), ('inception', '0800-01-01T00:00:00Z'), ('monarchyname', 'Danish Realm')]
[('monarchy', 'http://www.wikidata.org/entity/Q756617'), ('inception', '0800-01-01T00:00:00Z'), ('monarchyname', 'Danish Realm')]
[('monarchy', 'http://www.wikidata.org/entity/Q756

20

### Which are the top-3 monarchies with the earliest inception and still existing?

In [99]:
# Which are the top-3 monarchies with the earliest inception and still existing?
queryString = """
SELECT *
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P571 ?inception .

    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    FILTER NOT EXISTS {?monarchy wdt:P576 ?demolished_date} 
    FILTER NOT EXISTS {?monarchy wdt:P31 wd:Q19595246} 
} 
ORDER BY (?inception)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q787336'), ('inception', '0201-01-01T00:00:00Z'), ('monarchyname', 'Roman Catholic Diocese of Metz')]
[('monarchy', 'http://www.wikidata.org/entity/Q42834'), ('inception', '0395-01-18T00:00:00Z'), ('monarchyname', 'Western Roman Empire (395-476 AD)')]
[('monarchy', 'http://www.wikidata.org/entity/Q384237'), ('inception', '0569-01-01T00:00:00Z'), ('monarchyname', 'duchy of Asti')]


3

In [100]:
# Which are the top-3 monarchies with the earliest inception and still existing?
# [('p', 'http://www.wikidata.org/prop/direct/P576'), ('o', '-221-01-01T00:00:00Z'), ('pname', 'dissolved, abolished or demolished date')]
queryString = """
SELECT DISTINCT ?monarchyname ?inception
WHERE { 

    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
    ?monarchy wdt:P571 ?inception .
    ?monarchy ?p ?date .
    # Labels
    ?monarchy <http://schema.org/name> ?monarchyname .
    ?p <http://schema.org/name> ?pname .
    FILTER(!(regex(str(?p), "http://www.wikidata.org/prop/direct/P576" ) ))
} 
ORDER BY ?inception
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchyname', 'Galactic Empire'), ('inception', '11584-01-01T00:00:00Z')]
[('monarchyname', 'Kushan Empire'), ('inception', '0030-01-01T00:00:00Z')]
[('monarchyname', 'Kingdom of Funan'), ('inception', '0068-01-01T00:00:00Z')]
[('monarchyname', 'Chen princedom'), ('inception', '0088-01-01T00:00:00Z')]
[('monarchyname', 'Bharshiva Dynasty'), ('inception', '0170-01-01T00:00:00Z')]
[('monarchyname', 'Champa'), ('inception', '0192-01-01T00:00:00Z')]
[('monarchyname', 'Kingdom of Araba'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Shule Kingdom'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Early history of Kedah'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Langkasuka'), ('inception', '0200-01-01T00:00:00Z')]
[('monarchyname', 'Roman Catholic Diocese of Metz'), ('inception', '0201-01-01T00:00:00Z')]
[('monarchyname', 'Ghassanids'), ('inception', '0220-01-01T00:00:00Z')]
[('monarchyname', 'Shu Han'), ('inception', '0221-01-01T00:00:00Z

20